## Create "Model Topper" using example

In [1]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications
#from keras import backend as K
from keras import callbacks

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 1

top_model_weights_path = 'bottleneck_xception_model.h5'
train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

In [3]:
available_train_files % 8

0

Useful metrics for model evaluation

In [4]:
# def precision(y_true, y_pred):
#     """Precision metric.

#     Only computes a batch-wise average of precision.

#     Computes the precision, a metric for multi-label classification of
#     how many selected items are relevant.
#     """
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision


# def recall(y_true, y_pred):
#     """Recall metric.

#     Only computes a batch-wise average of recall.

#     Computes the recall, a metric for multi-label classification of
#     how many relevant items are selected.
#     """
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

Create Generators

In [5]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

Found 7368 images belonging to 2 classes.
Found 3152 images belonging to 2 classes.


Load Standard Xception Model

In [6]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build My "Custom" Model

In [7]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(1, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
_________________________________________________________________


Freeze Exception Layers

In [8]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [9]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

In [11]:
#Fit 

epochs = 1

cb1 = [callbacks.ModelCheckpoint(
    'xception_binary_convos1_best.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]




# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb1)

watch_model.save('xception_binary_test.h5')

Epoch 1/1
921/921 [==============================] - 451s - loss: 0.4951 - binary_accuracy: 0.7644 - val_loss: 0.4554 - val_binary_accuracy: 0.7995


In [23]:
del watch_model

In [24]:
watch_model = load_model('xception_binary_test.h5')

In [25]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-3:]:
    layer.trainable = True

epochs = 1


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    'xception_binary_unfrozen_convos2_best.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('xception_binary_unfrozen_convos1_test.h5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

Epoch 1/1
921/921 [==============================] - 470s - loss: 0.4595 - binary_accuracy: 0.7871 - val_loss: 0.4219 - val_binary_accuracy: 0.8068


In [21]:
del watch_model

In [2]:
watch_model = load_model('xception_binary_unfrozen_convos1_test.h5')